# Sentinel-2 indices using Google Earth Engine

This demo works analogue to the "Demo_Sentinel2_indices" but uses the Google Earth Engine (GEE) API package for python. This allows to compute large amount of satellite (and other remote sensing) data without having to download the huge data sets. To be able to use this package a GEE account is required.

### Start a GEE session

The first time the "ee" package is used, you need to run ee.Authenticate(). This will open a window on the web where you have to log in with your GEE credentials. This will create an access token that needs to be pasted in the box that appeared below (this needs to be done at the beginning or when the kernel/session is restarted). To start the connection with your GEE account, you run ee.Initialize(). If you have only one project on your GEE account, you do not need to specify more. If you have multiple projects, you can specify with ee.Initialize(project=project-number). To find the project number, you have to log in to GEE online and click on the respective project.

In [1]:
# Load the packages
import ee       # GEE API package
import geemap   # package for interactive plotting->does not work on PyCharm

# Login with the GEE credentials and connect to your account
ee.Authenticate()   # needs to be done once in a while
ee.Initialize()     # starts the connection to your GEE account and allows you to use all the datasets you might have stored there

Enter verification code:  4/1AVMBsJhJELPLwg-_5dJ0SuK2zXRYtdzU6CKf85f7SRJp5eIkzPX4G2bGgjo



Successfully saved authorization token.


---

## Selecting and Visualizing Sentinel-2 Data

### Filter Sentinel-2 data by date

We select the Sentinel-2 data catalogue (ImageCollection) and filter it by a time frame where the 23. August 2024 falls into. After filtering, we can print the size of the newly generate ImageCollection. We see that there are over 20'000 images available in that time frame! This is because we did not define an area of interest to filter the Sentinel-2 collection and it returns all images available globally.

In [2]:
# Define the study period
startTime = '2024-08-22'
endTime = '2024-08-24'

# Filter the Sentinel-2 image collection by date
S2_images_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filter(ee.Filter.date(startTime, endTime))

# Check how many images are available
print('Number of images in study period:', S2_images_col.size().getInfo())

Number of images in study period: 24833


### Define an area of interest

There are multiple ways to specify an area of interest:<br>
- Define the geometry using coordinates (e.g., 4 corners of a square with the 5th coordinate=first coordinate to close the shape)
- Load an already existing shapefile and convert it using geemap.shp_to_ee(shapefile)

In [3]:
# Define an area of interest using a set of coordinates as a list
aoi = ee.Geometry.Polygon([[9.829207878112806,46.80626536296483],[9.88104961395265,46.80626536296483],[9.88104961395265,46.82638289500436],
                           [9.829207878112806,46.82638289500436],[9.829207878112806,46.80626536296483]])

In [38]:
# Initialize the map
Map = geemap.Map()

# Define how the basemap should be displayed
Map.set_center(9.85, 46.82, 13)
Map.add_basemap('TERRAIN')

# Define the visualization parameters for the aoi, such as color (as hex-code)
vis_params = {'color': 'FFD700',
              'pointSize': 3,
              'pointShape': 'circle',
              'width': 2,
              'lineType': 'solid',
              'lineType': 'solid'}

# Add the area of interest as polygon to the map
Map.addLayer(aoi, vis_params, 'Area of interest')

# Display the map
Map

Map(center=[46.82, 9.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Filter Sentinel-2 data by date and area of interest

After filtering the Sentinel-2 ImageCollection by date and area of interest, we are left with only one available image in the ImageCollection. If we increase the length of the time period, more images would be in the Collection.

In [4]:
# Filter the Sentinel-2 image collection by area

# import the Sentinel-2 ImageCollection, filter by date and area
S2_images_col_aoi = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filter(ee.Filter.date(startTime, endTime))\
    .filter(ee.Filter.bounds(aoi))

# Check how many images are available
print('Number of images in study period:', S2_images_col_aoi.size().getInfo())

Number of images in study period: 1


To display the image, we need to convert the ImageCollection (which is currently only one image) to a unique image. As we only have one Image in the Collection, we can simply select the first image using ImageCollection.first(). If there were multiple images in the Collection, we would need to decide how to reduce our Collection to a single image:
- .first(): use the first image in the Collection (earliest in the time range)
- reducers such as .median(), .mean() etc.: calculate the e.g., pixel-wise mean value of all available images in the Collection (mean value of all pixels with the same coordinates)
- .mosaic(): uses the newest pixel values available

In [5]:
S2_image = S2_images_col_aoi.first()
S2_image

In [37]:
# Add the filtered Sentinel-2 image to our map
# Initialize the map
Map = geemap.Map()

# Define how the basemap should be displayed
Map.set_center(9.85, 46.82, 9.5)
Map.add_basemap('TERRAIN')

# Define the visualization parameters for the aoi, such as color (as hex-code)
vis_params = {'color': 'FFD700',
              'pointSize': 3,
              'pointShape': 'circle',
              'width': 2,
              'lineType': 'solid',
              'lineType': 'solid'}

# Add the Sentinel-2 image
Map.addLayer(S2_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},'Sentinel-2 image')

# Add the area of interest as polygon to the map
Map.addLayer(aoi, vis_params, 'Area of interest')

# Display the map
Map

Map(center=[46.82, 9.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In the displayed map, we can see how big the extent of one Sentinel-2 image (also named "tile") is. The filters we used above searched through the Sentinel-2 Collection and output the one tile that our desired area of interest is incorporated in. As we are only interested in the data within our AOI, we can clip this area and discard all the data around it. This will save a lot of computational time and storage if we decide to download the data in the end.

In [9]:
# Clip the desired area
clippedImage = S2_image.clip(aoi);
clippedImage

In [13]:
# Add the filtered Sentinel-2 image to our map
# Initialize the map
Map = geemap.Map()

# Define how the basemap should be displayed
Map.set_center(9.85, 46.82, 13)
Map.add_basemap('TERRAIN')

# Add the Sentinel-2 image
Map.addLayer(clippedImage, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},'Sentinel-2 image clipped')

# Display the map
Map

Map(center=[46.82, 9.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

---

## Image Analysis using Spectral Indices

In [24]:
# Define functions for the indices NDVI, NDMI, NDWI

def calcNDVI(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))

def calcNDMI(image):
    return image.addBands(image.normalizedDifference(['B8', 'B11']).rename('NDMI'))

def calcNDWI(image):
    return image.addBands(image.normalizedDifference(['B3', 'B8']).rename('NDWI'))

In [29]:
# Use the defined functions to add the indices as new layers to the Sentinel-2 image
S2_new = calcNDWI(calcNDMI(calcNDVI(clippedImage)))
S2_new

In [36]:
# Initialize the map
Map = geemap.Map()

# Define how the basemap should be displayed
Map.set_center(9.85, 46.82, 13)
Map.add_basemap('TERRAIN')

# NDVI: From grey (low vegetation) to green (high vegetation)
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['C0C0C0', '008000'],
}

# NDMI: From yellow (low moisture) to blue (high moisture)
ndmi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['FFFF00', '0000FF'],
}

# NDWI: From gray (no water) to blue (water)
ndwi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['C0C0C0', '0000FF'],
}

# Add the Sentinel-2 image
Map.addLayer(clippedImage, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'Sentinel-2 image clipped')

# Add the indices as layers with their respective visualizations
Map.addLayer(S2_new.select('NDVI'), ndvi_vis, 'NDVI')
Map.addLayer(S2_new.select('NDMI'), ndmi_vis, 'NDMI')
Map.addLayer(S2_new.select('NDWI'), ndwi_vis, 'NDWI')

# Display the map
Map

Map(center=[46.82, 9.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…